In [3]:
!pip install flask-cors
!pip install pyngrok


  Using cached flask_cors-5.0.1-py3-none-any.whl.metadata (961 bytes)
Using cached flask_cors-5.0.1-py3-none-any.whl (11 kB)


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import google.generativeai as genai
import pandas as pd
import os
from google.colab import userdata

# Initialize Flask
app = Flask(__name__)
CORS(app, resources={r"/ask": {"origins": "http://127.0.0.1:5500"}})

# Dataset folder path (inside repo)
DATASET_FOLDER = './datasets'
os.makedirs(DATASET_FOLDER, exist_ok=True)

# Initialize the Gemini API client
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

if not GOOGLE_API_KEY:
    print("Error: Please set the 'GOOGLE_API_KEY' secret in Colab.")
else:
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel(model_name='gemini-2.0-flash')

    # Define the folder path in Google Drive where your CSV files are stored
    folder_path = '/content/drive/My Drive/Hackathon'  # Change this to your folder path

    # Get the list of all CSV files in the folder
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    # Read and summarize each dataset
    dataset_summaries = []

    for file_name in csv_files:
        file_path = os.path.join(folder_path, file_name)
        try:
            df = pd.read_csv(file_path)
            summary = {
                "file": file_name,
                "shape": df.shape,
                "columns": df.columns.tolist(),
                "data_types": df.dtypes.astype(str).to_dict(),
                "missing_values": df.isnull().sum().to_dict(),
                "describe": df.describe(include='all').to_dict()
            }
            dataset_summaries.append(summary)
        except FileNotFoundError as e:
            print(f"Error: File not found: {file_path}")
        except Exception as e:
            print(f"Error reading file {file_name}: {e}")

    # Define the route to handle user queries
    @app.route('/ask', methods=['POST', 'OPTIONS'])
    def ask():
        if request.method == 'OPTIONS':
            return jsonify({'message': 'CORS preflight successful'}), 200
        elif request.method == 'POST':
            user_query = request.json.get('query', '')
            if not user_query:
                return jsonify({'error': 'Query is required'}), 400
            try:
                if GOOGLE_API_KEY:
                    corpus_prompt = f"""
         You're an AI chatbot assistant helping a restaurant merchant analyze their sales data and provide insights and recommendations.

        Below are summaries of the merchant's sales datasets:

        {dataset_summaries}

        The merchant asked: "{user_query}"

        Respond in a clear, helpful, and concise way. Get straight to the point and avoid unnecessary explanations.
        Do not show any calculations to the merchant.

        Focus on answering the question directly.
        Do not mention any backend processes, column names, files, or code logic.
        Keep the response informative but minimal.

        Give a direct, confident answer — no introductions, no step-by-step explanations, no backend references such as files, column names, variables, calculations, or methods.
        Do not explain how you are calculating or getting the data.Do not tell how you determine,analyse or calculate and etc,be straight to the answer.
        """
                    response = model.generate_content(corpus_prompt, generation_config={"temperature": 0.2})
                    return jsonify({'response': response.text}), 200
                else:
                    return jsonify({'error': 'Google API key not configured'}), 500
            except Exception as e:
                print(f"Error processing query: {e}")
                return jsonify({'error': str(e)}), 500

    if __name__ == '__main__':
        from pyngrok import ngrok
        NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)  # Replace with your actual ngrok authtoken
        try:
            public_url = ngrok.connect(5000)
            print(f"Flask app is live at {public_url}")
            app.run(host='0.0.0.0', port=5000, debug=False)
        except Exception as e:
            print(f"Error starting ngrok: {e}")
